In [4]:
import os
import torch
from torch import nn
from poutyne import Model, CSVLogger
from poutyne.framework import ModelCheckpoint, EarlyStopping, plot_history
import numpy as np
import torchmetrics
from datetime import datetime
import sys
import pandas as pd
from custom_lib.custom_models.basic_nn import NeuralNetwork
from custom_lib.data_prep import data_transformation_pipeline, data_loader
import matplotlib as plt
import torchvision.models as models
import time


In [5]:
# Tuneable Params
lr = 1e-3

data_dir = "data"

# Define a model name (e.g., "model1")
model_name = "custom_reduction_1_b0"

save_logs = True

epochs = 15

image_size = 224
rotate_angle=None
horizontal_flip_prob=None
brightess_contrast=None
gaussian_blur=None
normalize=False
seed = 42



In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [7]:
train_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=True)
test_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=False)
val_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=False)

train_loader , val_loader, test_loader, num_classes = data_loader(data_dir, 
                                                     train_transform=train_transform,
                                                     test_transform=test_transform,
                                                     val_transform=val_transform,
                                                     seed=seed
                                                     )



Train size: 6177, Validation size: 772, Test size: 773


In [5]:
# If Else statement to determine if the user has passed a custom model or a prebuilt model.
# If the model_name contains the word custom, the code extracts the version letter and number
# and passes the proper configuration to the model
if ("custom" in model_name):
    from custom_lib.custom_models.custom_eff_net import define_custom_eff_net
    import re

    efficient_net_config = {
        # tuple of width multiplier, depth multiplier, resolution, and Survival Prob for
        # each efficientnet version
        "b0" : (1.0, 1.0, 224, 0.2),
        "b1" : (1.0, 1.1, 240, 0.2),
        "b2" : (1.1, 1.2, 260, 0.3),
        "b3" : (1.2, 1.4, 300, 0.3),
        "b4" : (1.4, 1.8, 380, 0.4),
        "b5" : (1.6, 2.2, 456, 0.4),
        "b6" : (1.8, 2.6, 528, 0.5),
        "b7" : (2.0, 3.1, 600, 0.5)
    }

    model = define_custom_eff_net(efficient_net_config=efficient_net_config, num_classes=num_classes, model_name=model_name, device=device)

else:
    model_class = getattr(models, model_name, None)

    if model_class is None:
        raise ValueError(f"Model '{model_name}' is not available in torchvision.models.")

    # Initialize the model
    model = model_class(pretrained=True)











# Compound scaling factors for efficient-net family.


# 6. Wrap the model with Poutyne
poutyne_model = Model(
    model,
    optimizer=torch.optim.Adam(model.parameters(), lr=lr),
    loss_function=nn.CrossEntropyLoss(),
    batch_metrics=['accuracy'],
    device=device
)

In [9]:
from efficientnet_pytorch import EfficientNet

# Load the pretrained model (example for EfficientNet-B0)
pretrained_model = EfficientNet.from_pretrained('efficientnet-b0')

model.load_state_dict(pretrained_model.state_dict(), strict=False)

Loaded pretrained weights for efficientnet-b0


_IncompatibleKeys(missing_keys=['features.0.conv.weight', 'features.0.batch_norm.weight', 'features.0.batch_norm.bias', 'features.0.batch_norm.running_mean', 'features.0.batch_norm.running_var', 'features.1.depthwise_conv.conv.weight', 'features.1.depthwise_conv.batch_norm.weight', 'features.1.depthwise_conv.batch_norm.bias', 'features.1.depthwise_conv.batch_norm.running_mean', 'features.1.depthwise_conv.batch_norm.running_var', 'features.1.conv1k.weight', 'features.1.conv1k.bias', 'features.1.convk1.weight', 'features.1.convk1.bias', 'features.1.se.se.1.weight', 'features.1.se.se.1.bias', 'features.1.se.se.3.weight', 'features.1.se.se.3.bias', 'features.1.pointwise_conv.conv.weight', 'features.1.pointwise_conv.batch_norm.weight', 'features.1.pointwise_conv.batch_norm.bias', 'features.1.pointwise_conv.batch_norm.running_mean', 'features.1.pointwise_conv.batch_norm.running_var', 'features.1.batch_norm.weight', 'features.1.batch_norm.bias', 'features.1.batch_norm.running_mean', 'features

In [10]:

callbacks = None

# if save_logs == True:
#     # Callback: Save the best model based on validation accuracy
#     checkpoint = ModelCheckpoint(f"{results_dir}/best_model.pth", monitor='val_loss', mode='min', save_best_only=True)

#     # Callback: Stop training early if validation accuracy doesn't improve for 5 epochs
#     early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5)

#     # Set up the logger
#     csv_logger = CSVLogger(f"{results_dir}/training_logs.csv")

#     callbacks = [checkpoint, early_stopping, csv_logger]

start_time = time.time()
# 7. Train the model
history = poutyne_model.fit_generator(train_loader, val_loader, epochs=epochs, verbose=True,
                            callbacks = callbacks)
end_time = time.time()

run_time = end_time - start_time

print(f"Model training took {run_time / 60} minutes")


Epoch: 1/1 Train steps: 194 Val steps: 13 1m56.00s loss: 0.747324 acc: 67.864659 val_loss: 0.444689 val_acc: 82.383420
Model training took 1.9333831667900085 minutes


In [11]:
# 8. Evaluate the model
test_metrics = poutyne_model.evaluate_generator(test_loader)
print("Test metrics:", test_metrics)

Test steps: 13 31.04s test_loss: 0.521435 test_acc: 81.759379                                  
Test metrics: (0.5214351822765494, 81.75937906736544)


False